In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("demo").getOrCreate()

24/09/23 09:45:26 WARN Utils: Your hostname, manu-pc resolves to a loopback address: 127.0.1.1; using 192.168.0.110 instead (on interface wlp58s0)
24/09/23 09:45:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/23 09:45:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("../data/employees.csv", header='True', inferSchema="True")
df.show()


+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|employee_id|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|          NULL|       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|          NULL|       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|          NULL|       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|          NULL|       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|          NULL|       201|           20|


24/09/23 09:45:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


WITH RECURSIVE employee_hierarchy AS (
    SELECT id, name, manager_id, 0 AS level
    FROM employee
    WHERE manager_id IS NULL  -- Start from the top-level manager (e.g., CEO)

    UNION ALL

    SELECT e.id, e.name, e.manager_id, eh.level + 1
    FROM employee e
    INNER JOIN employee_hierarchy eh ON e.manager_id = eh.id
)

SELECT id, name, manager_id, level
FROM employee_hierarchy
ORDER BY level, name;


In [10]:
df.columns

['employee_id',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'COMMISSION_PCT',
 'MANAGER_ID',
 'DEPARTMENT_ID']

## Trying to get the employee name along with manager name 

In [11]:
query = """WITH RECURSIVE employee_hierarchy AS (
    SELECT employee_id, FIRST_NAME, MANAGER_ID, 0 AS level
    FROM employee
    WHERE manager_id IS NULL 

    UNION ALL

    SELECT e.employee_id, e.FIRST_NAME, e.MANAGER_ID, eh.level + 1
    FROM employee e
    INNER JOIN employee_hierarchy eh ON e.MANAGER_ID = eh.employee_id
)

SELECT employee_id, FIRST_NAME, MANAGER_ID, level
FROM employee_hierarchy
ORDER BY level, FIRST_NAME;
"""

df.createOrReplaceTempView("employee")

spark.sql(query).show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'employee_hierarchy'.(line 1, pos 15)

== SQL ==
WITH RECURSIVE employee_hierarchy AS (
---------------^^^
    SELECT employee_id, FIRST_NAME, MANAGER_ID, 0 AS level
    FROM employee
    WHERE manager_id IS NULL 

    UNION ALL

    SELECT e.employee_id, e.FIRST_NAME, e.MANAGER_ID, eh.level + 1
    FROM employee e
    INNER JOIN employee_hierarchy eh ON e.MANAGER_ID = eh.employee_id
)

SELECT employee_id, FIRST_NAME, MANAGER_ID, level
FROM employee_hierarchy
ORDER BY level, FIRST_NAME;


In [4]:
select 

FIRST_NAME, 
case when MANAGER_ID='NULL' then "HOD" else el
FIRST_NAME as mgr_name

+-------+---------+--------+-----+----------+
|OrderID|ProductID|Quantity|Price| OrderDate|
+-------+---------+--------+-----+----------+
|      1|      101|       3|  100|2023-01-01|
|      2|      102|       1|  200|2023-01-02|
|      3|      101|       2|  100|2023-01-03|
|      4|      103|       5|   50|2023-01-04|
+-------+---------+--------+-----+----------+



In [5]:
df.createOrReplaceTempView("employees")

In [8]:
from unittest.util import safe_repr


# query = "select * from employees;"  # working
query =  """
select e.employee_id, e.first_name, m.first_name as manager_name
from employees e
left join employees m on e.manager_id = m.employee_id;
"""

spark.sql(query).show()

+-----------+----------+------------+
|employee_id|first_name|manager_name|
+-----------+----------+------------+
|        198|    Donald|       Kevin|
|        199|   Douglas|       Kevin|
|        200|  Jennifer|       Neena|
|        201|   Michael|      Steven|
|        202|       Pat|     Michael|
|        203|     Susan|       Neena|
|        204|   Hermann|       Neena|
|        205|   Shelley|       Neena|
|        206|   William|     Shelley|
|        100|    Steven|        NULL|
|        101|     Neena|      Steven|
|        102|       Lex|      Steven|
|        103| Alexander|         Lex|
|        104|     Bruce|   Alexander|
|        105|     David|   Alexander|
|        106|     Valli|   Alexander|
|        107|     Diana|   Alexander|
|        108|     Nancy|       Neena|
|        109|    Daniel|       Nancy|
|        110|      John|       Nancy|
+-----------+----------+------------+
only showing top 20 rows



In [10]:
df.select("employee_id","first_name").show()

+-----------+----------+
|employee_id|first_name|
+-----------+----------+
|        198|    Donald|
|        199|   Douglas|
|        200|  Jennifer|
|        201|   Michael|
|        202|       Pat|
|        203|     Susan|
|        204|   Hermann|
|        205|   Shelley|
|        206|   William|
|        100|    Steven|
|        101|     Neena|
|        102|       Lex|
|        103| Alexander|
|        104|     Bruce|
|        105|     David|
|        106|     Valli|
|        107|     Diana|
|        108|     Nancy|
|        109|    Daniel|
|        110|      John|
+-----------+----------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import col

df.alias("e")\
    .join(df.alias('m'), col("e.manager_id")==col("m.employee_id"), "left")\
    .select(col("e.employee_id"), col("e.first_name"), col("m.first_name")).show()

+-----------+----------+----------+
|employee_id|first_name|first_name|
+-----------+----------+----------+
|        198|    Donald|     Kevin|
|        199|   Douglas|     Kevin|
|        200|  Jennifer|     Neena|
|        201|   Michael|    Steven|
|        202|       Pat|   Michael|
|        203|     Susan|     Neena|
|        204|   Hermann|     Neena|
|        205|   Shelley|     Neena|
|        206|   William|   Shelley|
|        100|    Steven|      NULL|
|        101|     Neena|    Steven|
|        102|       Lex|    Steven|
|        103| Alexander|       Lex|
|        104|     Bruce| Alexander|
|        105|     David| Alexander|
|        106|     Valli| Alexander|
|        107|     Diana| Alexander|
|        108|     Nancy|     Neena|
|        109|    Daniel|     Nancy|
|        110|      John|     Nancy|
+-----------+----------+----------+
only showing top 20 rows



In [12]:
# udf function trial fro commo practice

from pyspark.sql.functions import udf 

def salary_category(salary):
    if salary<5000:
        return "low salaried"
    elif salary < 10000:
        return "mid salaried"
    else:
        return "High salaried"
    

salary_udf = udf(salary_category)

df.withColumn("salary category", salary_udf(col("salary"))).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---------------+
|employee_id|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|salary category|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|          NULL|       124|           50|   low salaried|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|          NULL|       124|           50|   low salaried|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|          NULL|       101|           10|   low salaried|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|          NULL|       100|           20|  High salaried|
|        202|

In [16]:
# without udf
from pyspark.sql.functions import col,when

df.withColumn("salary category", 
              when(col('salary') < 5000, "low salaried")\
                .when(col("salary")< 10000, "mid salaried")\
                    .otherwise("high slaried")).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---------------+
|employee_id|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|salary category|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|          NULL|       124|           50|   low salaried|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|          NULL|       124|           50|   low salaried|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|          NULL|       101|           10|   low salaried|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|          NULL|       100|           20|   high slaried|
|        202|

In [23]:
df.withColumn("salary_category", when(((col("salary") <5000) & (col("job_id")=="SH_CLERK")), "lowsalaried and cleark")).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+--------------------+
|employee_id|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|     salary_category|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+--------------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|          NULL|       124|           50|lowsalaried and c...|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|          NULL|       124|           50|lowsalaried and c...|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|          NULL|       101|           10|                NULL|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|          NULL|       100|           20|

In [17]:
df.groupBy("department_id").agg({"salary":"sum"}).show()

+-------------+-----------+
|department_id|sum(salary)|
+-------------+-----------+
|           20|      19000|
|           40|       6500|
|          100|      51608|
|           10|       4400|
|           50|      85600|
|           70|      10000|
|           90|      58000|
|           60|      28800|
|          110|      20308|
|           30|      24900|
+-------------+-----------+



In [29]:
# trying windows function
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

win_spec = Window.partitionBy("department_id")

df.withColumn("total salary", sum("salary").over(win_spec)).orderBy(col("department_id")).show()


+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+------------+
|employee_id|FIRST_NAME|  LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|total salary|
+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+------------+
|        200|  Jennifer|     Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|          NULL|       101|           10|        4400|
|        201|   Michael|  Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|          NULL|       100|           20|       19000|
|        202|       Pat|        Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|          NULL|       201|           20|       19000|
|        114|       Den|   Raphaely|DRAPHEAL|515.127.4561|07-DEC-02|  PU_MAN| 11000|          NULL|       100|           30|       24900|
|        115| Alexander|       Kho